In [1]:
# -*- coding: utf-8 -*-
"""
Colab Notebook: AI Imagery Pipeline Test (Initial Stages)

This notebook simulates the initial stages of the AI Imagery Pipeline:
1. User Input Collection (Custom & Task-Specific Modes) with Social Media Platform
2. Simplified JSON Output Generation (including image metadata)
3. Enhanced JSON Parsing & Validation (Simulated Client/Server)
4. Image Analysis using VLM (Simulated/LLM with Pydantic/Instructor) - Step 5
5. Marketing Strategy Generation (Simulated/LLM with Pydantic/Instructor) - Step 6 (Staged Reasoning: List of Niches -> Goals, No Pools, Retries, Token Usage)
6. Includes setup instructions for OpenRouter LLM integration.
7. Processing triggered by running the final cell directly.
8. Halts processing if Image Evaluation API call fails.
9. Saves final JSON and input image (if applicable) with unique filenames.
"""

'\nColab Notebook: AI Imagery Pipeline Test (Initial Stages)\n\nThis notebook simulates the initial stages of the AI Imagery Pipeline:\n1. User Input Collection (Custom & Task-Specific Modes) with Social Media Platform\n2. Simplified JSON Output Generation (including image metadata)\n3. Enhanced JSON Parsing & Validation (Simulated Client/Server)\n4. Image Analysis using VLM (Simulated/LLM with Pydantic/Instructor) - Step 5\n5. Marketing Strategy Generation (Simulated/LLM with Pydantic/Instructor) - Step 6 (Staged Reasoning: List of Niches -> Goals, No Pools, Retries, Token Usage)\n6. Includes setup instructions for OpenRouter LLM integration.\n7. Processing triggered by running the final cell directly.\n8. Halts processing if Image Evaluation API call fails.\n9. Saves final JSON and input image (if applicable) with unique filenames.\n'

In [2]:
!pip install openai pydantic instructor python-dotenv tenacity Pillow -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.6/345.6 kB 5.9 MB/s eta 0:00:00


In [3]:
# @title Setup: Import Libraries and Define Tasks/Platforms/Pools
import ipywidgets as widgets
from IPython.display import display, clear_output
import json
from PIL import Image
import io
import os
# Used for OpenRouter integration
# !pip install openai # Uncomment and run this line if openai library is not installed
from google.colab import drive
from dotenv import load_dotenv
import traceback
import base64 # For potential image encoding
from typing import List, Optional, Dict, Any # For Pydantic models
import random # For fallback simulation diversity
import time # For adding timestamps to logs
import pathlib # For handling paths and extensions
import datetime # For timestamped filenames

try:
    # Use the new client-based import structure
    from openai import OpenAI, APIConnectionError, RateLimitError, APIStatusError
    from pydantic import BaseModel, Field, field_validator # Import Pydantic
    import instructor # Import instructor
    from tenacity import RetryError # Import RetryError for specific checking
except ImportError:
    print("Required libraries (openai>=1.0.0, pydantic, instructor, tenacity) not found.")
    print("Please install them (`pip install -U openai pydantic instructor tenacity`) to use OpenRouter and Pydantic.")
    OpenAI = None
    BaseModel = None # Set to None if import fails
    instructor = None
    RetryError = None # Set to None if import fails
    openai = None # Keep compatibility with older checks if needed, but prioritize OpenAI class


# Define the available task types for Task-Specific Mode
TASK_TYPES = [
    'Select Task...',
    '1. Product Photography',
    '2. Promotional Graphics & Announcements',
    '3. Store Atmosphere & Decor',
    '4. Menu Spotlights',
    '5. Cultural & Community Content',
    '6. Recipes & Food Tips',
    '7. Brand Story & Milestones',
    '8. Behind the Scenes Imagery'
]

# Define target social media platforms with resolutions (Mandatory)
# Using a dictionary: Key = Display Name, Value = Resolution Dict
SOCIAL_MEDIA_PLATFORMS = {
    'Select Platform...': None, # Default Placeholder
    'Instagram Post (1:1 Square)': {'width': 1080, 'height': 1080, 'aspect_ratio': '1:1'},
    'Instagram Story/Reel (9:16 Vertical)': {'width': 1080, 'height': 1920, 'aspect_ratio': '9:16'},
    'Facebook Post (Mixed)': {'width': 1200, 'height': 630, 'aspect_ratio': '1.91:1 or 1:1'}, # FB is flexible, common link preview size
    'Pinterest Pin (2:3 Vertical)': {'width': 1000, 'height': 1500, 'aspect_ratio': '2:3'},
    'Xiaohongshu (Red Note) (3:4 Vertical)': {'width': 1080, 'height': 1440, 'aspect_ratio': '3:4'},
    # Add other relevant platforms if needed
}
# Get the list of display names for the dropdown widget
PLATFORM_DISPLAY_NAMES = list(SOCIAL_MEDIA_PLATFORMS.keys())


# --- Option Pools REMOVED (Mostly) ---
# Pools might still be used for fallback simulation or inspiration in prompts
TASK_GROUP_POOLS = {
    "product_focus": {
        "audience": ["Foodies/Bloggers", "Local Residents", "Health-Conscious Eaters", "Budget-Conscious Diners", "Young Professionals (25-35)"],
        "niche": ["Casual Dining", "Fine Dining", "Cafe/Coffee Shop", "Ethnic Cuisine", "Takeaway/Delivery Focused"],
        "objective": ["Create Appetite Appeal", "Showcase Quality/Freshness", "Promote Specific Menu Item", "Increase Online Orders/Reservations"],
        "voice": ["Mouth-watering & Descriptive", "Sophisticated & Elegant", "Fresh & Vibrant", "Authentic & Honest"]
    },
    "default": { # Fallback pool
        "audience": ["Local Residents", "Young Professionals (25-35)", "Families with Children", "Foodies/Bloggers"],
        "niche": ["Casual Dining", "Cafe/Coffee Shop", "Takeaway/Delivery Focused"],
        "objective": ["Increase Brand Awareness", "Drive Foot Traffic", "Increase Engagement on Social Media"],
        "voice": ["Friendly & Casual", "Warm & Welcoming", "Authentic & Honest"]
    }
}
# Helper function to get the appropriate pool based on task type string
def get_pools_for_task(task_type_str):
    if not task_type_str: return TASK_GROUP_POOLS["default"]
    if task_type_str.startswith('1.') or task_type_str.startswith('4.'): return TASK_GROUP_POOLS["product_focus"]
    # Add mappings for other task groups if defined
    return TASK_GROUP_POOLS["default"] # Fallback


print("Libraries imported, Task Types, and Platforms defined.")
if OpenAI and BaseModel and instructor and RetryError:
    print("Ensure 'openai' (>=1.0.0), 'pydantic', 'instructor', and 'tenacity' libraries are installed.")
else:
    print("WARNING: One or more required libraries (openai, pydantic, instructor, tenacity) not imported. LLM/Pydantic features will be skipped.")


Libraries imported, Task Types, and Platforms defined.
Ensure 'openai' (>=1.0.0), 'pydantic', 'instructor', and 'tenacity' libraries are installed.


In [4]:
# @title Step 1: Mode Selection
mode_selection = widgets.RadioButtons(
    options=['Custom Mode', 'Task-Specific Mode'],
    description='Select Mode:',
    disabled=False
)

display(mode_selection)

RadioButtons(description='Select Mode:', options=('Custom Mode', 'Task-Specific Mode'), value='Custom Mode')

In [5]:
# @title Step 2: Input Widgets Setup
# --- Common Widgets ---
platform_selection = widgets.Dropdown(
    options=PLATFORM_DISPLAY_NAMES, # Use display names for options
    value=PLATFORM_DISPLAY_NAMES[0], # Default to placeholder
    description='*Platform Target:', # Added asterisk for mandatory
    disabled=False,
    style={'description_width': 'initial'} # Adjust width if needed
)

prompt_input = widgets.Textarea(
    value='',
    placeholder='Enter your text prompt here (describe style, composition, setting, etc.)...',
    description='Prompt:',
    layout={'height': '100px', 'width': '95%'}
)

image_upload = widgets.FileUpload(
    accept='image/*',  # Accept image files
    multiple=False,   # Allow only single file upload
    description='Upload Image Ref:'
)

image_instruction = widgets.Textarea(
    value='',
    placeholder='(Optional) Provide brief instructions for the uploaded image (e.g., "Use the burger as the main subject", "Describe the style and setting", "Extract the text"). If empty, only the main subject will be identified.',
    description='Image Instruction:',
    layout={'height': '60px', 'width': '95%'}
)

# --- Task-Specific Widgets ---
task_selection = widgets.Dropdown(
    options=TASK_TYPES,
    value=TASK_TYPES[0], # Default to placeholder
    description='*Task Type:', # Added asterisk for mandatory
    disabled=False,
    style={'description_width': 'initial'}
)

branding_elements_input = widgets.Textarea(
    value='',
    placeholder='(Optional) Describe branding elements (e.g., "Use warm colors like #F5A623", "Include logo placeholder top-right", "Font: Playful Sans-serif")...',
    description='Branding:',
    layout={'height': '80px', 'width': '95%'}
)

task_description_input = widgets.Textarea(
    value='',
    placeholder='(Optional) Enter content specific to the task (e.g., "Promo Text: 2-for-1 Coffee!", "Menu Item: Signature Pasta", "Milestone: 5 Year Anniversary")...',
    description='Task Content:',
    layout={'height': '80px', 'width': '95%'}
)

marketing_audience = widgets.Text(value='', placeholder='(Optional) e.g., Young professionals, families', description='Target Audience:')
marketing_objective = widgets.Text(value='', placeholder='(Optional) e.g., Increase engagement, drive sales', description='Objective:')
marketing_voice = widgets.Text(value='', placeholder='(Optional) e.g., Playful, sophisticated, casual', description='Voice:')
marketing_niche = widgets.Text(value='', placeholder='(Optional) e.g., Vegan cafe, fine dining', description='Niche:')

marketing_goals_box = widgets.VBox([
    widgets.HTML("<b>Optional Marketing Goals:</b>"),
    marketing_audience,
    marketing_objective,
    marketing_voice,
    marketing_niche
])

# --- Layout Containers ---
# Note: platform_selection is added to both VBox containers
custom_mode_widgets = widgets.VBox([
    widgets.HTML("<h3>Custom Mode Inputs:</h3>"),
    platform_selection, # Added Platform Selection
    prompt_input,
    image_upload,
    image_instruction
])

task_specific_mode_widgets = widgets.VBox([
    widgets.HTML("<h3>Task-Specific Mode Inputs:</h3>"),
    platform_selection, # Added Platform Selection
    task_selection,     # Task selection is mandatory for this mode
    prompt_input, # Re-use prompt input
    image_upload, # Re-use image upload
    image_instruction, # Re-use image instruction
    branding_elements_input,
    task_description_input,
    marketing_goals_box
])

# --- Output Widget ---
output_area = widgets.Output()

# --- Display Logic ---
input_container = widgets.VBox([]) # Empty container to hold dynamic widgets

def on_mode_change(change):
    """Handles switching between Custom and Task-Specific modes."""
    # No need to clear output here as it's displayed separately later
    # with output_area:
    #    clear_output()
    if change['new'] == 'Custom Mode':
        input_container.children = [custom_mode_widgets]
    elif change['new'] == 'Task-Specific Mode':
        input_container.children = [task_specific_mode_widgets]
    else:
        input_container.children = []

# Initialize display based on default mode
on_mode_change({'new': mode_selection.value})

# Observe changes in mode selection
mode_selection.observe(on_mode_change, names='value')

print("Input widgets configured. Select a mode above to see the relevant inputs.")
# Display ONLY the input container here
display(input_container)

Input widgets configured. Select a mode above to see the relevant inputs.


In [12]:
# @title Mount Google Drive and Set Input File Path
# This cell mounts your Google Drive to access files stored there.
# It will prompt you for authorization the first time you run it.

try:
    drive.mount('/content/drive')
    print("✅ Google Drive mounted successfully.")

    # --- Define Input File Path on Google Drive ---
    # ** IMPORTANT: Update the filename if needed **
    DRIVE_BASE_PATH = '/content/drive/MyDrive/AI Imagery Marketing Tool/Colab Notebook'

    # Check if the directory and file exist after mounting
    if not os.path.isdir(DRIVE_BASE_PATH):
        print(f"❌ Error: Google Drive directory not found: {DRIVE_BASE_PATH}")
        print("   Please ensure the path is correct and Drive is mounted properly.")
    else:
        print(f"✅ Input file path set to: {DRIVE_BASE_PATH}")

except Exception as e:
    print(f"❌ An error occurred during Google Drive mounting or path setting: {e}")

Mounted at /content/drive
✅ Google Drive mounted successfully.
✅ Input file path set to: /content/drive/MyDrive/AI Imagery Marketing Tool/Colab Notebook
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Google Drive mounted successfully.
✅ Input file path set to: /content/drive/MyDrive/AI Imagery Marketing Tool/Colab Notebook


In [7]:
# @title Step 3: LLM Setup (OpenRouter - Optional but Recommended for Real Implementation)

# --- Instructions ---
# 1. Get an API Key: Sign up at https://openrouter.ai to get your free API key.
# 2. Secure Your Key: In Colab, it's best practice to store secrets securely.
#    Go to the "Secrets" tab (key icon on the left panel) and add a new secret named 'OPENROUTER_API_KEY'
#    with your actual API key as the value. Enable "Notebook access".
# 3. Run this cell: It will attempt to load the key and configure the OpenAI client.

# --- Configuration Code ---
# --- IMPORTANT ---
# Set your OpenRouter API key (using .env file or environment variable recommended)
dotenv_path = "/content/drive/MyDrive/AI Imagery Marketing Tool/Colab Notebook/colab_secrets/.env"

if os.path.exists(dotenv_path):
    load_dotenv(dotenv_path=dotenv_path)
    print(f"Loaded .env file from path: {dotenv_path}")
else:
    print(f"Error: .env file not found at path: {dotenv_path}")

OPENROUTER_API_KEY = os.getenv("OPENROUTER_API_KEY")
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY_3")


# Select LLM service provider
LLM_SERVICE_PROVIDER = "OpenRouter" # or "Gemini" or "openai" or "OpenRouter"
IMG_EVAL_MODEL = "openai/gpt-4.1-mini" # Make sure this model supports image input via API gemini-2.5-pro-exp-03-25 openai/gpt-4.1-mini google/gemini-2.5-flash-preview
STRATEGY_MODEL = "openai/gpt-4.1-mini" # Keep mini for testing, but note limitations gemini-2.5-pro-exp-03-25 openai/gpt-4.1-mini google/gemini-2.5-flash-preview

client = None
instructor_client = None # Initialize instructor client
MAX_LLM_RETRIES = 1 # Define max retries for LLM calls

if OpenAI and instructor: # Check if both libraries are available
  if OPENROUTER_API_KEY or GEMINI_API_KEY:
    if LLM_SERVICE_PROVIDER == "OpenRouter":
      BASE_API_URL = "https://openrouter.ai/api/v1"
      BASE_API_KEY = OPENROUTER_API_KEY
    elif LLM_SERVICE_PROVIDER == "Gemini":
      BASE_API_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"
      BASE_API_KEY = GEMINI_API_KEY

    try:
              # Initialize the base OpenAI client
              base_client = OpenAI(
                  api_key=BASE_API_KEY,
                  base_url=BASE_API_URL,
                  max_retries=MAX_LLM_RETRIES, # Configure retries here
                  # Pass headers during initialization if needed by OpenRouter or specific models
                  # default_headers={
                  #    "HTTP-Referer": "http://localhost:8888", # Replace with your app URL/Identifier
                  #    "X-Title": "Colab Pipeline Tester" # Replace with your app name/Identifier
                  # }
              )
              # Patch the client with instructor (NO max_retries argument here)
              instructor_client = instructor.patch(base_client)
              print(f"OpenAI client configured for OpenRouter and patched with Instructor (max_retries={MAX_LLM_RETRIES}).")
    except Exception as e:
              print(f"Error initializing OpenAI client or patching with Instructor: {e}")
              client = None # Ensure client is None if initialization fails
              instructor_client = None
  elif not OPENROUTER_API_KEY:
      print("⚠️ OpenRouter API Key not found.")
      # OPENROUTER_API_KEY = input("Enter your OpenRouter API Key: ")
  elif not GEMINI_API_KEY:
      print("⚠️ Gemini API Key not found.")
      # GEMINI_API_KEY = input("Enter your Gemini API Key: ")
  else:
      print("OpenAI client not configured for OpenRouter (API Key missing).")
      print("LLM-dependent steps will use simulated data only.")
else:
    print("OpenAI library (>=1.0.0) and/or Instructor library not available. Cannot configure OpenRouter.")
    client = None # Ensure client is None if library is missing
    instructor_client = None

Loaded .env file from path: /content/drive/MyDrive/AI Imagery Marketing Tool/Colab Notebook/colab_secrets/.env
OpenAI client configured for OpenRouter and patched with Instructor (max_retries=1).


In [8]:
# @title Step 4: Define Processing Functions

# --- Processing Functions ---

def validate_inputs(mode, inputs):
    """
    Performs basic validation on the raw inputs (Simulates Client-Side).
    Checks mandatory fields before attempting to generate JSON.
    """
    # --- MANDATORY CHECK: Social Media Platform ---
    # Check if the selected platform is the placeholder/default value
    if not inputs.get('platform') or inputs['platform'] == PLATFORM_DISPLAY_NAMES[0]:
        return False, "Client-Side Validation Error: A target Social Media Platform must be selected."

    if mode == 'Custom Mode':
        # Check if at least one of prompt or image is provided
        if not inputs.get('prompt') and not inputs.get('image_details'):
            return False, "Client-Side Validation Error: Custom Mode requires either a text prompt or an uploaded image."
    elif mode == 'Task-Specific Mode':
        # --- MANDATORY CHECK: Task Type ---
        # Check if a valid task type is selected (not the placeholder)
        if not inputs.get('task_type') or inputs['task_type'] == TASK_TYPES[0]:
             return False, "Client-Side Validation Error: Task-Specific Mode requires selecting a valid Task Type."
        # Add more task-specific mandatory checks here if needed (e.g., promo text for task 2)
        # Example: Check for task_content if task is Promotional Graphics
        # if inputs.get('task_type') == '2. Promotional Graphics & Announcements' and not inputs.get('task_content'):
        #     return False, "Client-Side Validation Error: Promotional Graphics task requires Task Content (Promo Text)."

    # Check if image details exist if image was supposedly uploaded
    if inputs.get('image_widget_has_value') and not inputs.get('image_details'):
         # This indicates an internal error during image detail extraction
         return False, "Client-Side Validation Error: Image was uploaded but details could not be extracted."

    return True, "Client-Side Validation: Inputs seem valid for the selected mode."

def generate_initial_json(mode, inputs):
    """Creates the structured JSON output based on validated inputs."""
    image_ref_data = None
    if inputs.get('image_details'):
        image_ref_data = {
            "filename": inputs['image_details'].get('filename'),
            "content_type": inputs['image_details'].get('content_type'),
            "size_bytes": inputs['image_details'].get('size_bytes'),
            "instruction": inputs.get('image_instruction', None), # Instruction is separate from details
            # Store image content if needed for VLM call later
            "image_content_base64": inputs.get('image_content_base64', None)
        }

    # Capture marketing goals even if partially filled or None/empty string
    marketing_goals_data = {
        "target_audience": inputs.get('mkt_audience') if inputs.get('mkt_audience') else None,
        "objective": inputs.get('mkt_objective') if inputs.get('mkt_objective') else None,
        "voice": inputs.get('mkt_voice') if inputs.get('mkt_voice') else None,
        "niche": inputs.get('mkt_niche') if inputs.get('mkt_niche') else None
    }
    # Check if *all* are None before setting the whole object to None
    # We want to keep partially filled goals for the strategist agent
    if all(v is None for v in marketing_goals_data.values()):
        marketing_goals_data = None # Set to None only if completely empty

    # Get selected platform details (including resolution)
    selected_platform_key = inputs.get('platform')
    platform_details = SOCIAL_MEDIA_PLATFORMS.get(selected_platform_key) # Get the dict value

    output_json = {
      "request_details": {
        "mode": mode.lower().replace(" ", "_"), # e.g., "custom_mode"
        "task_type": inputs.get('task_type', None) if mode == 'Task-Specific Mode' else None,
        "target_platform": { # Store platform name and details
            "name": selected_platform_key,
            "resolution": platform_details # Contains width, height, aspect_ratio or None
        }
      },
      "user_inputs": {
        "prompt": inputs.get('prompt', None),
        "image_reference": image_ref_data, # Use the structured image data or None
        "branding_elements": inputs.get('branding', None),
        "task_description": inputs.get('task_content', None),
        "marketing_goals": marketing_goals_data # Store user-provided goals (potentially partial or None)
      },
      "processing_context": {
          "initial_json_valid": None, # To be filled later
          "image_analysis_result": None, # To store analysis output
          "suggested_marketing_strategies": None, # To store strategist output
          "llm_call_usage": {} # To store token usage
      }
    }

    return output_json

def parse_and_validate_json(generated_json):
    """
    Simulates parsing and structural validation of the generated JSON (Simulates Server-Side).
    Checks if the received JSON structure is as expected.
    """
    try:
        # Check top-level keys
        if not all(k in generated_json for k in ["request_details", "user_inputs", "processing_context"]):
            raise ValueError("Missing required top-level keys (request_details, user_inputs, processing_context).")

        request_details = generated_json.get("request_details", {})
        user_inputs = generated_json.get("user_inputs", {})

        # Validate request_details
        mode = request_details.get("mode").replace("-","_")
        platform_info = request_details.get("target_platform") # Now an object
        platform_name = platform_info.get("name") if isinstance(platform_info, dict) else None

        if not mode or not platform_info or not platform_name:
             raise ValueError("Missing essential keys in request_details: mode or target_platform object/name.")
        if platform_name == PLATFORM_DISPLAY_NAMES[0]: # Check placeholder name wasn't passed
             raise ValueError("Invalid target_platform name found in request_details.")
        # Check if resolution exists and is a dictionary (or None if placeholder was somehow passed)
        resolution = platform_info.get("resolution")
        if resolution is not None and not isinstance(resolution, dict):
             raise ValueError("Invalid target_platform resolution format found in request_details.")

        # --- FIXED MODE VALIDATION ---
        # Check if the mode string is one of the expected values
        valid_modes = ["custom_mode", "task_specific_mode"]
        if mode not in valid_modes:
            # This error message will now correctly trigger if the mode string is unexpected
            raise ValueError(f"Unknown mode '{mode}' found in request_details. Expected one of {valid_modes}.")

        # Validate user_inputs based on the now confirmed valid mode
        if mode == "custom_mode":
            # Custom mode needs either prompt or image_reference
            if user_inputs.get("prompt") is None and user_inputs.get("image_reference") is None:
                raise ValueError("Custom mode JSON requires 'prompt' or 'image_reference' in user_inputs.")
        elif mode == "task_specific_mode":
             # Task-specific mode needs a task_type
             task_type = request_details.get("task_type")
             if not task_type:
                 raise ValueError("Task-specific mode JSON requires 'task_type' in request_details.")
             if task_type == TASK_TYPES[0]: # Check placeholder task type wasn't passed
                 raise ValueError("Invalid task_type value found in request_details.")
             # Add checks for mandatory task_description based on task_type if needed
             # Example:
             # if request_details.get('task_type') == '2. Promotional Graphics & Announcements' and not user_inputs.get('task_description'):
             #     raise ValueError("Promotional Graphics task JSON requires 'task_description'.")
        # No else needed here because we already validated the mode string above

        # Validate image_reference structure if it exists
        image_ref = user_inputs.get("image_reference")
        if image_ref is not None:
            if not isinstance(image_ref, dict):
                 raise ValueError("user_inputs.image_reference must be an object (dict).")
            if not all(k in image_ref for k in ["filename", "content_type", "size_bytes"]):
                 raise ValueError("user_inputs.image_reference is missing required keys (filename, content_type, size_bytes).")
            # Could add type checks for size_bytes (int), content_type (str), etc.

        # Validate marketing_goals structure if it exists
        mkt_goals = user_inputs.get("marketing_goals")
        # It's okay for marketing_goals to be None initially
        if mkt_goals is not None and not isinstance(mkt_goals, dict):
             raise ValueError("user_inputs.marketing_goals must be an object (dict) or None.")
             # Could check keys within marketing_goals if needed and not None

        generated_json["processing_context"]["initial_json_valid"] = True
        return True, "Server-Side Validation: JSON structure seems valid."
    except Exception as e:
        generated_json["processing_context"]["initial_json_valid"] = False
        return False, f"Server-Side Validation Error: {e}"

In [9]:
# @title Step 5: Define Pydantic Models (Image Analysis & Marketing Strategy)

# Define the Pydantic model only if BaseModel was imported successfully
if BaseModel:
    class ImageAnalysisResult(BaseModel):
        """Structured result of the image analysis."""
        main_subject: str = Field(..., description="The single, primary subject of the image (e.g., 'Gourmet Burger', 'Latte Art', 'Restaurant Interior'). Should be concise.")
        secondary_elements: Optional[List[str]] = Field(None, description="(Only if requested by instruction) List other notable objects or elements present.")
        setting_environment: Optional[str] = Field(None, description="(Only if requested by instruction) Describe the background or setting.")
        style_mood: Optional[str] = Field(None, description="(Only if requested by instruction) Describe the inferred visual style, mood, or atmosphere.")
        extracted_text: Optional[str] = Field(None, description="(Only if requested by instruction) Extract any visible text from the image.")
        suggested_keywords: Optional[List[str]] = Field(None, description="(Optional) List 3-5 relevant keywords for tagging.")

    # Model for Stage 1: Identifying relevant niches
    class RelevantNicheList(BaseModel):
        """Identifies a list of relevant F&B niches for the given context."""
        relevant_niches: List[str] = Field(..., description="A list of 3-5 diverse but highly relevant F&B niches based on the input context (e.g., ['Ethnic Cuisine (Thai)', 'Casual Dining', 'Takeaway/Delivery Focused']). Prioritize niches directly related to the image subject or task description.")
        # justification: Optional[str] = Field(None, description="Brief overall justification for choosing these niches.") # Optional justification

    # Model for Stage 2: Generating goals based on a chosen niche
    class MarketingGoalSetStage2(BaseModel):
        """Represents a set of marketing goals (audience, objective, voice) aligned with a specific niche."""
        target_audience: str = Field(..., description="Specific target audience group, generated based on context and relevant to the predetermined niche.")
        target_objective: str = Field(..., description="The primary marketing objective for this asset, generated based on context and relevant to the predetermined niche/task.")
        target_voice: str = Field(..., description="The desired brand voice or tone, generated based on context and relevant to the predetermined niche/audience.")

    class MarketingStrategyOutputStage2(BaseModel):
         """Container for N suggested marketing goal combinations (audience, objective, voice)."""
         strategies: List[MarketingGoalSetStage2] = Field(..., description="A list of N diverse and strategically sound marketing goal combinations (audience, objective, voice) aligned with a predetermined niche.")

    # Final structure stored in JSON context (includes niche)
    class MarketingGoalSetFinal(BaseModel):
        """Represents a complete set of marketing goals for a creative direction."""
        target_audience: str
        target_niche: str
        target_objective: str
        target_voice: str


    print("Pydantic models 'ImageAnalysisResult', 'RelevantNicheList', 'MarketingGoalSetStage2', 'MarketingStrategyOutputStage2', and 'MarketingGoalSetFinal' defined.")
else:
    ImageAnalysisResult = None
    RelevantNicheList = None # New model name
    MarketingGoalSetStage2 = None
    MarketingStrategyOutputStage2 = None
    MarketingGoalSetFinal = None # Need to handle this in fallback too
    print("Pydantic not available, cannot define models.")

Pydantic models 'ImageAnalysisResult', 'RelevantNicheList', 'MarketingGoalSetStage2', 'MarketingStrategyOutputStage2', and 'MarketingGoalSetFinal' defined.


In [10]:
# @title Step 6: Perform Image Evaluation (Simulated/LLM with Pydantic)

def perform_image_evaluation(generated_json):
    """
    Performs image analysis using a VLM via OpenRouter (with Instructor/Pydantic)
    if configured, otherwise simulates the analysis. Focuses on the main subject
    unless instructed otherwise. Includes retries and token usage.
    Returns a status code: 'SUCCESS', 'API_ERROR', 'SIMULATED_NO_API', 'SIMULATED_FALLBACK', 'NO_IMAGE'.
    """
    image_ref = generated_json.get("user_inputs", {}).get("image_reference")
    analysis_result = None
    status_message = "No image provided for evaluation."
    usage_info = None # To store token usage
    status_code = 'NO_IMAGE' # Default status

    if not image_ref: # No image uploaded
        generated_json["processing_context"]["image_analysis_result"] = None
        return status_message, None, usage_info, status_code

    filename = image_ref.get("filename")
    instruction = image_ref.get("instruction") # User's specific instruction
    content_type = image_ref.get("content_type")
    size = image_ref.get("size_bytes")
    image_content_base64 = image_ref.get("image_content_base64") # Get base64 content if stored

    task_type = generated_json.get("request_details", {}).get("task_type", "N/A")
    platform = generated_json.get("request_details", {}).get("target_platform", {}).get("name", "N/A")

    status_prefix = f"Image '{filename}' ({content_type}, {size} bytes): "

    # Determine analysis scope based on instruction
    analysis_scope_instruction = "Your primary goal is to identify the `main_subject` concisely."
    if instruction:
        analysis_scope_instruction = f"Follow the user's instruction: '{instruction}'. If the instruction asks for more details (like style, setting, secondary elements, text extraction), provide them concisely in the relevant fields of the response model. Otherwise, focus ONLY on the `main_subject`."
    else:
        analysis_scope_instruction += " No specific user instruction was provided."


    # --- Attempt LLM/VLM Call via OpenRouter using Instructor ---
    # Use the globally configured 'instructor_client' variable from Step 3
    if instructor_client and ImageAnalysisResult: # Check if instructor client and Pydantic model are available
        print(f"(Attempting VLM call via OpenRouter/Instructor for image '{filename}'...)")
        try:
            # --- VLM Prompt Engineering ---
            # Construct messages list for multimodal input
            prompt_messages = []
            user_content = [
                {
                    "type": "text",
                    "text": f"""Analyze the provided image for an F&B marketing task.
                            Context: Task='{task_type}', Target Platform='{platform}'.
                            {analysis_scope_instruction}

                            Provide the analysis based on the requested `ImageAnalysisResult` response model. Be concise."""
                }
            ]

            # ** Add Image Data **
            if image_content_base64:
                 user_content.append({
                     "type": "image_url",
                     "image_url": {
                         # Use base64 data URI format
                         "url": f"data:{content_type};base64,{image_content_base64}"
                      }
                 })
            else:
                 # If base64 content isn't available, we cannot make a true VLM call
                 raise ValueError("Image content (base64) is missing for VLM analysis.")


            prompt_messages = [
                {
                    "role": "system",
                    "content": "You are an expert visual analyst for F&B marketing. Provide concise, structured analysis matching the requested Pydantic model."
                },
                {
                    "role": "user",
                    "content": user_content # Pass the list containing text and image data
                }
            ]

            print(f"  Sending request to model: {IMG_EVAL_MODEL}")
            completion = instructor_client.chat.completions.create(
                model=IMG_EVAL_MODEL,
                response_model=ImageAnalysisResult,
                messages=prompt_messages,
                temperature=0.3,
                max_tokens=400,
            )
            analysis_result_object = completion

            raw_response = getattr(completion, '_raw_response', None)
            if raw_response and hasattr(raw_response, 'usage') and raw_response.usage:
                usage_info = raw_response.usage.model_dump()
                print(f"  Token Usage (Image Eval): {usage_info}")
            else:
                 print("  Token usage data not directly available from response object.")

            analysis_result = analysis_result_object.model_dump()
            print(f"  Successfully received and validated Pydantic response from VLM.")
            status_message = status_prefix + "Analysis complete (via VLM/Instructor)."
            status_code = 'SUCCESS' # API call was successful


        except ValueError as ve:
             print(f"  ERROR preparing VLM call: {ve}")
             status_message = status_prefix + f"Analysis skipped ({ve}). Falling back to simulation."
             analysis_result = simulate_image_evaluation_fallback(instruction).model_dump()
             status_code = 'SIMULATED_FALLBACK' # Fallback due to internal error

        # Catch specific OpenAI/HTTP errors if needed
        except (APIConnectionError, RateLimitError, APIStatusError) as api_error:
             print(f"  ERROR: API call failed: {api_error}")
             status_message = status_prefix + f"Analysis failed ({type(api_error).__name__}). Falling back to simulation."
             analysis_result = simulate_image_evaluation_fallback(instruction).model_dump()
             status_code = 'API_ERROR' # Indicate API call failed
        except Exception as e: # Catch other potential errors (e.g., validation errors from Instructor)
            if RetryError and isinstance(e, RetryError):
                 print(f"  ERROR: LLM call failed after {MAX_LLM_RETRIES + 1} attempts. Cause: {e.last_attempt.exception()}")
                 status_message = status_prefix + f"Analysis failed after retries ({e.last_attempt.exception()}). Falling back to simulation."
                 status_code = 'API_ERROR' # Indicate API call failed after retries
            else:
                 print(f"  ERROR during OpenRouter/Instructor API call: {e}")
                 status_message = status_prefix + f"Analysis failed (API/Validation Error: {e}). Falling back to simulation."
                 status_code = 'API_ERROR' # Indicate other API/Validation error
            print(traceback.format_exc())
            analysis_result = simulate_image_evaluation_fallback(instruction).model_dump()

    else: # Fallback if OpenRouter client or Pydantic is not configured
         print("(OpenRouter/Instructor client or Pydantic not configured, using basic simulation for image evaluation)")
         analysis_result = simulate_image_evaluation_fallback(instruction).model_dump()
         status_message = status_prefix + "Analysis simulated (No API Key / Library)."
         status_code = 'SIMULATED_NO_API' # Indicate simulation due to config

    # Store the result (dict) in the main JSON
    generated_json["processing_context"]["image_analysis_result"] = analysis_result
    # Store usage info if available
    if usage_info:
        generated_json["processing_context"]["llm_call_usage"]["image_eval"] = usage_info

    return status_message, analysis_result, usage_info, status_code # Return status code

def simulate_image_evaluation_fallback(instruction):
    """
    Provides a simulated structured analysis (as a Pydantic object)
    when LLM call is not possible.
    """
    if not ImageAnalysisResult: # Check if Pydantic model is defined
         # Return a simple dict if Pydantic model isn't available
         return {"error": "Pydantic model not defined", "main_subject": "Simulated Subject (No Pydantic)"}

    data = {
            "main_subject": "Identified Subject (Simulated e.g., Burger)",
            "secondary_elements": None,
            "setting_environment": None,
            "style_mood": None,
            "extracted_text": None,
            "suggested_keywords": ["simulated", "generic", "subject", "food", "fallback"]
        }
    if instruction:
         # Simulate instruction influencing the subject
        data["main_subject"] = f"Subject based on instruction (Simulated)"
        # Optionally simulate filling other fields if instruction implies it
        if "style" in instruction.lower():
            data["style_mood"] = "Simulated style based on instruction"
        if "setting" in instruction.lower():
             data["setting_environment"] = "Simulated setting based on instruction"
        if "text" in instruction.lower():
             data["extracted_text"] = "Simulated extracted text"
        if "element" in instruction.lower() or "object" in instruction.lower():
             data["secondary_elements"] = ["Simulated element 1", "Simulated element 2"]


    # Create and return a Pydantic object
    try:
      return ImageAnalysisResult(**data)
    except Exception as e:
        print(f"Error creating fallback Pydantic object: {e}")
        # Return dict on error creating Pydantic obj
        return {"error": "Fallback object creation failed", "main_subject": "Simulated Subject (Error)"}




In [11]:
# @title Step 7: Generate Marketing Strategies (Simulated/LLM with Pydantic - Staged Reasoning) - MODIFIED STEP

def generate_marketing_strategies(generated_json, num_strategies: int = 3):
    """
    Generates N diverse marketing strategy combinations using a STAGED LLM approach
    if the user hasn't provided complete goals.
    Stage 1: Identify a list of relevant niches.
    Stage 2: Generate combinations based on identified niches and other context.
    Includes retries and token usage.
    """
    user_goals = generated_json.get("user_inputs", {}).get("marketing_goals") # Can be None or dict
    # Check if user provided all goals (handle None case and empty strings)
    user_goals_complete = False
    if user_goals:
         user_goals_complete = all(user_goals.get(k) for k in ['target_audience', 'objective', 'voice', 'niche']) # Checks for non-empty values

    status_message = "User provided complete marketing goals. Skipping strategy generation."
    suggested_strategies = None # Will hold list of dicts
    usage_info_stage1 = None
    usage_info_stage2 = None
    status_code = 'SKIPPED' # Default status

    if user_goals_complete:
        print(status_message)
        # Store the user's single complete goal set as the only strategy
        generated_json["processing_context"]["suggested_marketing_strategies"] = [user_goals]
        return status_message, [user_goals], None, None, status_code # Return None for usage

    # Proceed if goals are incomplete or not provided
    status_message = f"Generating {num_strategies} marketing strategy suggestions..."
    print(status_message) # Print status immediately

    # --- Extract relevant context for the strategist ---
    task_type = generated_json.get("request_details", {}).get("task_type", "N/A")
    platform = generated_json.get("request_details", {}).get("target_platform", {}).get("name", "N/A")
    user_prompt = generated_json.get("user_inputs", {}).get("prompt") # Can be None
    task_description = generated_json.get("user_inputs", {}).get("task_description") # Can be None
    image_analysis = generated_json.get("processing_context", {}).get("image_analysis_result") # Can be None or dict
    image_subject = "N/A"
    if isinstance(image_analysis, dict):
        # Avoid passing error messages as subject
        if "error" not in image_analysis:
             image_subject = image_analysis.get("main_subject", "N/A")
        else:
             image_subject = "Analysis Failed"

    # --- Select Task-Specific Pools (for Stage 2 fallback/inspiration) ---
    task_pools = get_pools_for_task(task_type)
    audience_pool = task_pools.get("audience", TASK_GROUP_POOLS["default"]["audience"])
    niche_pool_inspiration = task_pools.get("niche", TASK_GROUP_POOLS["default"]["niche"]) # For fallback/inspiration only
    objective_pool = task_pools.get("objective", TASK_GROUP_POOLS["default"]["objective"])
    voice_pool = task_pools.get("voice", TASK_GROUP_POOLS["default"]["voice"])

    # --- STAGE 1: Identify Relevant Niches ---
    identified_niches = [] # Now a list
    stage1_status = "Starting Niche Identification..."
    print(f"  Stage 1: {stage1_status}")
    stage1_status_code = 'INIT'
    num_niches_to_find = random.randint(3, 5) # Target 3-5 niches

    # Use user-provided niche if available as the *only* niche
    user_provided_niche = (user_goals or {}).get('target_niche')
    if user_provided_niche:
        identified_niches = [user_provided_niche]
        stage1_status = f"Using user-provided niche: '{identified_niches[0]}'."
        print(f"    Status: {stage1_status}")
        stage1_status_code = 'USER_PROVIDED'

        stage1_status = f"Using user-provided niche: '{identified_niches[0]}'."
        print(f"    Status: {stage1_status}")
        stage1_status_code = 'USER_PROVIDED'
    elif instructor_client and RelevantNicheList: # Check dependencies
        try:
            print(f"    (Attempting LLM call for {num_niches_to_find} Niche Identifications...)")
            niche_system_prompt = f"""You are an expert F&B market analyst. Your task is to identify {num_niches_to_find} diverse but MOST relevant F&B niches for the given context. Consider the image subject, task description, and task type. Prioritize the most logical fits based on the context. Output ONLY the Pydantic `RelevantNicheList` object containing the list of niche names."""
            niche_user_prompt = f"""Identify {num_niches_to_find} diverse but relevant F&B niches for this context:
Task Type: {task_type or 'N/A'}
Task-Specific Content/Description: {task_description or 'Not Provided'}
Identified Image Subject: {image_subject or 'Not Provided / Not Applicable'}
User-Provided Niche (Use if provided): {(user_goals or {}).get('target_niche') or 'Not Provided'}

Determine the {num_niches_to_find} best `relevant_niches` based on the context, especially the image subject."""

            completion_niche = instructor_client.chat.completions.create(
                model=STRATEGY_MODEL, # Or a more capable model
                response_model=RelevantNicheList, # Use the new list model
                messages=[
                    {"role": "system", "content": niche_system_prompt},
                    {"role": "user", "content": niche_user_prompt}
                ],
                temperature=0.5, # Allow some diversity in niche selection
                max_tokens=200,
            )
            niche_list_object = completion_niche

            # Try to access usage data
            raw_response_niche = getattr(completion_niche, '_raw_response', None)
            if raw_response_niche and hasattr(raw_response_niche, 'usage') and raw_response_niche.usage:
                usage_info_stage1 = raw_response_niche.usage.model_dump()
                print(f"    Token Usage (Niche ID): {usage_info_stage1}")
            else:
                 print("    Token usage data not directly available from niche response object.")


            identified_niches = niche_list_object.relevant_niches # Get the list
            stage1_status = f"Niches identified via LLM: {identified_niches}"
            print(f"    Status: {stage1_status}")
            stage1_status_code = 'SUCCESS'

        except Exception as e:
            print(f"    ERROR during Niche Identification LLM call: {e}")
            stage1_status = "Niche Identification LLM call failed. Falling back to simulation."
            print(f"    Status: {stage1_status}")
            # Fallback niche selection
            identified_niches = random.sample(niche_pool_inspiration, min(num_niches_to_find, len(niche_pool_inspiration)))
            stage1_status_code = 'API_ERROR'


    else: # Fallback if client/Pydantic not available
        stage1_status = "Niche Identification skipped (Client/Pydantic unavailable). Using simulation."
        print(f"  Stage 1: {stage1_status}")
        identified_niches = random.sample(niche_pool_inspiration, min(num_niches_to_find, len(niche_pool_inspiration)))
        stage1_status_code = 'SIMULATED_NO_API'

    if not identified_niches: # Ensure we always have at least one niche for stage 2
         print("    WARNING: No niches identified or simulated, using default.")
         identified_niches = [random.choice(niche_pool_inspiration)]
         stage1_status += " (Used default niche as fallback)"

    print(f"  Using Niches: {identified_niches} for strategy generation.")

    # --- STAGE 2: Generate Goal Combinations based on Identified Niches ---
    stage2_status = "Starting Goal Combination Generation..."
    print(f"  Stage 2: {stage2_status}")
    stage2_status_code = 'INIT'
    suggested_strategies = [] # Initialize list for final combined strategies

    if instructor_client and MarketingStrategyOutputStage2 and MarketingGoalSetStage2 and MarketingGoalSetFinal: # Check dependencies
        try:
            print("    (Attempting LLM call for Goal Combinations...)")
            # --- Strategist Prompt Engineering (Stage 2 - Refined) ---
            system_prompt_stage2 = f"""You are an expert F&B Marketing Strategist. Your goal is to generate {num_strategies} diverse and strategically sound marketing goal combinations (audience, objective, voice).
For each combination:
1.  Select ONE niche from the provided 'Relevant Niches List'. Aim to use different niches from the list across the {num_strategies} combinations for diversity, unless the user provided a specific niche (in which case, use that one).
2.  Generate a fitting `target_audience`, `target_objective`, and `target_voice` that logically align with the **chosen niche** for that specific combination and the overall context (task type, image subject).
**Handling User Input:**
- If the user provided a value for audience, objective, or voice, incorporate **relevant variations** of that theme into the generated combinations where it logically fits the chosen niche for that strategy. For example, if the user objective is 'increase sales', generated objectives could be 'Drive Short-Term Sales', 'Boost Online Orders', 'Increase Average Order Value', etc., all related to increasing sales and fitting the chosen niche. Do not just copy the user input verbatim unless it's the best fit.
- For goals the user did *not* provide, generate appropriate values based on the context, chosen niche, and user-provided goals (if any).
Ensure the {num_strategies} generated combinations are distinct from each other and make sense. Try to use different niches from the provided list across the strategies for diversity.
Output the result matching the `MarketingStrategyOutputStage2` model containing a list of exactly {num_strategies} `MarketingGoalSetStage2` objects (audience, objective, voice ONLY). **Ensure each item in the list is a valid JSON object.**"""

            # User prompt now includes the list of identified niches
            user_prompt_context_stage2 = f"""Generate {num_strategies} diverse marketing strategy combinations (audience, objective, voice) for the following F&B task. For each strategy, choose a relevant niche from the list provided below and ensure the generated goals align with it.
Task Type: {task_type or 'N/A'}
Target Platform: {platform or 'N/A'}
User's General Prompt: {user_prompt or 'Not Provided'}
Task-Specific Content/Description: {task_description or 'Not Provided'}
Identified Image Subject: {image_subject or 'Not Provided / Not Applicable'}

**Relevant Niches List (Choose one niche from this list for each strategy):** {identified_niches}

User-Provided Goals (Use as strong guidelines/constraints for generated values):
Audience: {(user_goals or {}).get('target_audience') or 'Not Provided'}
Objective: {(user_goals or {}).get('target_objective') or 'Not Provided'}
Voice: {(user_goals or {}).get('target_voice') or 'Not Provided'}
(User-provided Niche '{user_provided_niche}' influenced the Relevant Niches List above).

Generate {num_strategies} complete, diverse, and strategically relevant `MarketingGoalSetStage2` combinations (audience, objective, voice only). Ensure logical consistency with the chosen niche from the list and overall context. **Adhere strictly to the output format (list of JSON objects).**"""

            prompt_messages_stage2 = [
                {"role": "system", "content": system_prompt_stage2},
                {"role": "user", "content": user_prompt_context_stage2}
            ]

            #llm_model = "openai/gpt-4o-mini" # Or a more capable model

            print(f"    Sending request to model: {STRATEGY_MODEL}")
            completion_stage2 = instructor_client.chat.completions.create(
                model=STRATEGY_MODEL,
                response_model=MarketingStrategyOutputStage2, # Use Stage 2 Pydantic model
                messages=prompt_messages_stage2,
                temperature=0.7,
                max_tokens=1500, # Keep increased max_tokens
            )
            strategy_output_object_stage2 = completion_stage2

            # Try to access usage data
            raw_response_strat = getattr(completion_stage2, '_raw_response', None)
            if raw_response_strat and hasattr(raw_response_strat, 'usage') and raw_response_strat.usage:
                usage_info_stage2 = raw_response_strat.usage.model_dump()
                print(f"    Token Usage (Goal Combos): {usage_info_stage2}")
            else:
                 print("    Token usage data not directly available from goal combo response object.")

            # Convert the list of Pydantic objects to a list of dicts for storing
            # Add the assigned niche back into each strategy dict
            temp_suggested_strategies = []
            for i, strategy_stage2 in enumerate(strategy_output_object_stage2.strategies):
                strategy_dict = strategy_stage2.model_dump()
                # Assign niche - cycle through identified niches or use user's
                assigned_niche = user_provided_niche if user_provided_niche else identified_niches[i % len(identified_niches)]
                strategy_dict['target_niche'] = assigned_niche

                # *** Override other fields if user provided them ***
                if (user_goals or {}).get('target_audience'):
                    strategy_dict['target_audience'] = user_goals['target_audience'] # Or generate variation if needed
                if (user_goals or {}).get('target_objective'):
                    strategy_dict['target_objective'] = user_goals['target_objective'] # Or generate variation if needed
                if (user_goals or {}).get('target_voice'):
                    strategy_dict['target_voice'] = user_goals['target_voice'] # Or generate variation if needed

                # Ensure all keys are present before appending
                if all(k in strategy_dict for k in ['target_audience', 'target_niche', 'target_objective', 'target_voice']):
                     # Validate final dict with the complete model before appending
                     try:
                          final_strategy_obj = MarketingGoalSetFinal(**strategy_dict)
                          temp_suggested_strategies.append(final_strategy_obj.model_dump())
                     except Exception as pydantic_error:
                          print(f"    WARNING: Failed to validate final strategy structure: {strategy_dict}. Error: {pydantic_error}")
                else:
                     print(f"    WARNING: Incomplete strategy generated by LLM (missing keys before niche assignment): {strategy_stage2.model_dump()}")

            suggested_strategies = temp_suggested_strategies # Assign the final list


            # Validate if we got the requested number of strategies
            if len(suggested_strategies) != num_strategies:
                 print(f"    WARNING: LLM generated {len(suggested_strategies)} valid strategies, but {num_strategies} were requested.")
            else:
                 print(f"    Successfully received and validated {len(suggested_strategies)} marketing strategies from LLM.")
            stage2_status = "Goal combinations generated successfully (via LLM/Instructor)."
            stage2_status_code = 'SUCCESS'


        except APIConnectionError as e:
             print(f"    ERROR: Failed to connect to OpenRouter API: {e}")
             stage2_status = f"Goal Combination generation failed (Connection Error). Falling back to simulation."
             suggested_strategies = simulate_marketing_strategy_fallback_staged(user_goals, identified_niches, task_type, num_strategies)
             stage2_status_code = 'API_ERROR'
        except RateLimitError as e:
             print(f"    ERROR: OpenRouter Rate limit exceeded: {e}")
             stage2_status = f"Goal Combination generation failed (Rate Limit Error). Falling back to simulation."
             suggested_strategies = simulate_marketing_strategy_fallback_staged(user_goals, identified_niches, task_type, num_strategies)
             stage2_status_code = 'API_ERROR'
        except APIStatusError as e:
             print(f"    ERROR: OpenRouter API returned an error status: {e.status_code} - {e.response}")
             stage2_status = f"Goal Combination generation failed (API Status Error {e.status_code}). Falling back to simulation."
             suggested_strategies = simulate_marketing_strategy_fallback_staged(user_goals, identified_niches, task_type, num_strategies)
             stage2_status_code = 'API_ERROR'
        except Exception as e: # Catch other potential errors
            print(f"    ERROR during OpenRouter/Instructor API call for goal combinations: {e}")
            print(traceback.format_exc()) # Print full traceback for debugging
            stage2_status = f"Goal Combination generation failed (API/Validation Error: {e}). Falling back to simulation."
            suggested_strategies = simulate_marketing_strategy_fallback_staged(user_goals, identified_niches, task_type, num_strategies)
            stage2_status_code = 'API_ERROR'

    else: # Fallback if OpenRouter client or Pydantic is not configured
         print("    (OpenRouter/Instructor client or Pydantic not configured, using basic simulation for goal combinations)")
         suggested_strategies = simulate_marketing_strategy_fallback_staged(user_goals, identified_niches, task_type, num_strategies)
         stage2_status = "Goal combinations simulated (No API Key / Library)."
         stage2_status_code = 'SIMULATED_NO_API'

    print(f"  Stage 2: {stage2_status}")
    # Store the final result (list of dicts) in the main JSON
    generated_json["processing_context"]["suggested_marketing_strategies"] = suggested_strategies
    # Store usage info if available
    if usage_info_stage1: generated_json["processing_context"]["llm_call_usage"]["strategy_niche_id"] = usage_info_stage1
    if usage_info_stage2: generated_json["processing_context"]["llm_call_usage"]["strategy_goal_gen"] = usage_info_stage2

    # Determine overall status code
    if stage1_status_code == 'API_ERROR' or stage2_status_code == 'API_ERROR':
        overall_status_code = 'API_ERROR'
    elif stage1_status_code == 'SIMULATED_NO_API' or stage2_status_code == 'SIMULATED_NO_API':
         overall_status_code = 'SIMULATED_NO_API'
    elif stage1_status_code in ['USER_PROVIDED', 'SUCCESS'] and stage2_status_code == 'SUCCESS':
         overall_status_code = 'SUCCESS'
    else: # Handle other combinations if necessary
         overall_status_code = 'UNKNOWN'


    overall_status_message = f"Marketing strategies generated ({stage1_status.split('.')[0]}; {stage2_status.split('.')[0]})"
    # Return combined usage info (or individual if preferred)
    combined_usage = {**(usage_info_stage1 or {}), **(usage_info_stage2 or {})} # Simple merge, might overwrite keys if same name
    return overall_status_message, suggested_strategies, combined_usage, overall_status_code # Return status code

def simulate_marketing_strategy_fallback_staged(user_goals, identified_niches, task_type, num_strategies=3):
    """
    Provides N simulated marketing strategies (list of dicts) for the staged approach,
    using the predetermined niche list.
    """
    if not MarketingGoalSetFinal: # Check if Pydantic model is defined
        return [{"error": "Pydantic model not defined, cannot simulate fallback strategies."}] * num_strategies

    # Get task-specific pools for more relevant defaults (excluding niche)
    task_pools = get_pools_for_task(task_type)
    audience_pool = task_pools.get("audience", TASK_GROUP_POOLS["default"]["audience"])
    objective_pool = task_pools.get("objective", TASK_GROUP_POOLS["default"]["objective"])
    voice_pool = task_pools.get("voice", TASK_GROUP_POOLS["default"]["voice"])

    strategies = []
    used_combinations = set() # To ensure some diversity based on audience/objective/voice

    # Ensure identified_niches is a list and not empty
    if not isinstance(identified_niches, list) or not identified_niches:
        identified_niches = [random.choice(TASK_GROUP_POOLS["default"]["niche"])]


    for i in range(num_strategies):
        # Cycle through identified niches for diversity
        current_niche = identified_niches[i % len(identified_niches)]
        # Override niche if user provided one
        if (user_goals or {}).get('target_niche'):
             current_niche = user_goals['target_niche']

        current_goals = {
        "target_audience": (user_goals or {}).get('target_audience') or random.choice(audience_pool),
        "target_niche": current_niche, # Use the assigned/user niche
        "target_objective": (user_goals or {}).get('target_objective') or random.choice(objective_pool),
        "target_voice": (user_goals or {}).get('target_voice') or random.choice(voice_pool)
    }

        # Ensure uniqueness for simulation (simple approach)
        combination_tuple = tuple([current_goals["target_audience"], current_goals["target_objective"], current_goals["target_voice"], current_goals["target_niche"]])
        attempts = 0
        while combination_tuple in used_combinations and attempts < 10: # Avoid infinite loop
             # Try changing objective first for variation
             current_goals["target_objective"] = random.choice(objective_pool) + " (Alt)"
             combination_tuple = tuple([current_goals["target_audience"], current_goals["target_objective"], current_goals["target_voice"], current_goals["target_niche"]])
             attempts += 1

        if combination_tuple not in used_combinations:
            try:
                 # Validate with Pydantic before adding, even in fallback
                 validated_goals = MarketingGoalSetFinal(**current_goals)
                 strategies.append(validated_goals.model_dump())
                 used_combinations.add(combination_tuple)
            except Exception as e:
                 print(f"Error creating fallback strategy {i+1}: {e}")
                 strategies.append({"error": f"Fallback strategy {i+1} creation failed", "target_niche": current_niche})
                 used_combinations.add(combination_tuple) # Add even if failed to avoid infinite loop
        elif len(strategies) < num_strategies: # Add placeholder if uniqueness failed after attempts
             strategies.append({"error": "Could not generate unique fallback strategy", "target_niche": current_niche})


    # If we still don't have enough, add duplicates (less ideal)
    while len(strategies) < num_strategies:
         if strategies: # Check if strategies list is not empty
            strategies.append(strategies[0])
         else: # If even the first strategy failed
            strategies.append({"error": "Fallback failed completely", "target_niche": identified_niches[0]})

    return strategies


# --- Main Processing Function ---
def run_pipeline_processing():
    """Gathers inputs, validates, generates JSON, and runs processing steps."""
    # Add a timestamp to differentiate runs in logs
    run_timestamp_str = time.strftime("%Y%m%d_%H%M%S") # Timestamp for filenames
    print(f"\n--- Starting Run: {run_timestamp_str} ---")

    # --- MODIFICATION: Clear output area before starting the processing inside it ---
    clear_output(wait=True)
    # --- END MODIFICATION ---

    with output_area:
        # Using wait=True ensures it clears before new output appears
        # clear_output(wait=True) # Moved outside the 'with' block
        print(f"Processing request... (Run started at {run_timestamp_str})")
        final_json = None # Initialize
        total_usage = {} # To aggregate usage across steps
        saved_image_path_str = None # Initialize path for saved image

        try:
            # 1. Gather Inputs
            print("--- Step 1: Gathering Inputs ---")
            mode = mode_selection.value
            inputs = {
                'platform': platform_selection.value, # Get selected platform display name
                'prompt': prompt_input.value if prompt_input.value else None, # Store None if empty
                'image_widget_has_value': bool(image_upload.value) # Track if upload widget has data
            }
            print("  Inputs gathered (excluding image content).")

            # Handle image upload - Extract details AND content (Base64)
            uploaded_file_info = image_upload.value
            inputs['image_details'] = None # Initialize
            inputs['image_content_base64'] = None # Initialize
            image_content_bytes = None # Store bytes for saving later

            if uploaded_file_info:
                try:
                    # Get the first uploaded file's info (since multiple=False)
                    first_file_key = list(uploaded_file_info.keys())[0]
                    metadata = uploaded_file_info[first_file_key]['metadata']
                    image_content_bytes = uploaded_file_info[first_file_key]['content'] # Read bytes

                    inputs['image_details'] = {
                        'filename': metadata.get('name'),
                        'content_type': metadata.get('type'),
                        'size_bytes': metadata.get('size')
                    }
                    # Encode image content to Base64 for VLM API call
                    inputs['image_content_base64'] = base64.b64encode(image_content_bytes).decode('utf-8')
                    print(f"  Image '{metadata.get('name')}' processed.")

                except Exception as e:
                    print(f"  Error processing uploaded image details/content: {e}")
                    # Keep image_details as None if error occurs
                    inputs['image_details'] = None
                    inputs['image_content_base64'] = None
            else:
                print("  No image uploaded.")


            inputs['image_instruction'] = image_instruction.value if image_instruction.value else None


            if mode == 'Task-Specific Mode':
                inputs['task_type'] = task_selection.value # Get task type value
                inputs['branding'] = branding_elements_input.value if branding_elements_input.value else None
                inputs['task_content'] = task_description_input.value if task_description_input.value else None
                inputs['mkt_audience'] = marketing_audience.value if marketing_audience.value else None
                inputs['mkt_objective'] = marketing_objective.value if marketing_objective.value else None
                inputs['mkt_voice'] = marketing_voice.value if marketing_voice.value else None
                inputs['mkt_niche'] = marketing_niche.value if marketing_niche.value else None


            # 2. Validate Raw Inputs (Simulated Client-Side)
            print("\n--- Step 2: Client-Side Input Validation ---")
            is_valid, validation_msg = validate_inputs(mode, inputs)
            print(f"Status: {validation_msg}")
            if not is_valid:
                print(f"--- Run ENDED (Validation Failed): {time.strftime('%Y-%m-%d %H:%M:%S')} ---")
                return # Stop processing if basic inputs are invalid

            # 3. Generate Initial JSON
            print("\n--- Step 3: Generating Initial JSON ---")
            generated_json = generate_initial_json(mode, inputs)
            # Don't print the base64 content in the main output for brevity
            generated_json_display = json.loads(json.dumps(generated_json)) # Deep copy
            if generated_json_display.get("user_inputs",{}).get("image_reference"):
                 if "image_content_base64" in generated_json_display["user_inputs"]["image_reference"]:
                      del generated_json_display["user_inputs"]["image_reference"]["image_content_base64"]
            print(json.dumps(generated_json_display, indent=2))

            final_json = generated_json # Keep track of the latest JSON state

            # 4. Parse and Validate JSON Structure (Simulated Server-Side)
            print("\n--- Step 4: Server-Side JSON Validation ---")
            json_ok, json_validation_msg = parse_and_validate_json(generated_json)
            print(f"Status: {json_validation_msg}")
            # Update JSON with validation status (already done inside the function)
            if not json_ok:
                print("\nERROR: Server-side validation failed. Stopping processing.")
                print("\n--- Final JSON Output (Validation Failed) ---")
                # Display without base64
                final_json_display = json.loads(json.dumps(final_json)) # Deep copy
                if final_json_display.get("user_inputs",{}).get("image_reference"):
                     if "image_content_base64" in final_json_display["user_inputs"]["image_reference"]:
                          del final_json_display["user_inputs"]["image_reference"]["image_content_base64"]
                print(json.dumps(final_json_display, indent=2))
                print(f"--- Run ENDED (Validation Failed): {time.strftime('%Y-%m-%d %H:%M:%S')} ---")
                return


            # 5. Perform Image Evaluation
            print("\n--- Step 5: Image Evaluation ---")
            eval_status, analysis_result, eval_usage, eval_status_code = perform_image_evaluation(generated_json) # Get status code
            if eval_usage: total_usage["image_eval"] = eval_usage # Store usage
            print(f"Status: {eval_status}")
            # The analysis result (dict) is added to generated_json inside the function
            final_json = generated_json # Update final_json with analysis results

            # --- MODIFIED: Check if image evaluation failed due to API error ---
            if eval_status_code == 'API_ERROR':
                 print("\nERROR: Image Evaluation failed due to API error. Halting pipeline.")
                 print("\n--- Final JSON Output (Image Eval Failed) ---")
                 # Display without base64
                 final_json_display = json.loads(json.dumps(final_json)) # Deep copy
                 if final_json_display.get("user_inputs",{}).get("image_reference"):
                      if "image_content_base64" in final_json_display["user_inputs"]["image_reference"]:
                           del final_json_display["user_inputs"]["image_reference"]["image_content_base64"]
                 print(json.dumps(final_json_display, indent=2))
                 print(f"--- Run ENDED (Image Eval Failed): {time.strftime('%Y-%m-%d %H:%M:%S')} ---")
                 return # Stop processing

            # 6. Generate Marketing Strategies (Staged Approach)
            print("\n--- Step 6: Marketing Strategy Generation ---")
            # Define how many strategies to generate
            num_strategies_to_generate = 5 # Changed back to 5 as per user's previous output example
            strategy_status, suggested_strategies, strategy_usage, strategy_status_code = generate_marketing_strategies(generated_json, num_strategies=num_strategies_to_generate)
            # Store combined usage under a single key
            if strategy_usage: total_usage["strategy_gen_combined"] = strategy_usage
            print(f"Status: {strategy_status}")
            # Strategies (list of dicts) are added to generated_json inside the function

            final_json = generated_json # Update final_json with strategies

            # Update total usage in the final JSON
            final_json["processing_context"]["llm_call_usage"] = total_usage

            print("\n--- Final JSON Output (Before File Save) ---")
             # Display without base64
            final_json_display = json.loads(json.dumps(final_json)) # Deep copy
            if final_json_display.get("user_inputs",{}).get("image_reference"):
                 if "image_content_base64" in final_json_display["user_inputs"]["image_reference"]:
                      del final_json_display["user_inputs"]["image_reference"]["image_content_base64"]
            # The analysis result and strategies are already dicts here
            print(json.dumps(final_json_display, indent=2))

            # --- Step 7: Save Output Files ---
            print("\n--- Step 7: Saving Output Files ---")
            # MODIFIED: Use specified Google Drive path
            output_base_path = '/content/drive/MyDrive/AI Imagery Marketing Tool/Colab Notebook'
            output_folder_name = 'pipeline_upstream_outputs'
            output_dir = pathlib.Path(output_base_path) / output_folder_name
            # END MODIFICATION

            # Ensure the directory exists (create if not) - requires Drive mounted
            try:
                output_dir.mkdir(parents=True, exist_ok=True) # Create directory if it doesn't exist
                print(f"  Output directory: {output_dir}")
            except Exception as dir_error:
                print(f"  ERROR creating output directory '{output_dir}': {dir_error}")
                print("  Please ensure Google Drive is mounted and the path is accessible.")
                print("  Skipping file saving.")
                # Optionally end execution if saving is critical
                # print(f"--- Run ENDED (Directory Error): {time.strftime('%Y-%m-%d %H:%M:%S')} ---")
                # return

            # Create unique base filename using the run's timestamp string
            base_filename = f"output_{run_timestamp_str}"

            # Save Image if exists
            saved_image_filename = None
            if image_content_bytes and final_json.get("user_inputs", {}).get("image_reference"):
                try:
                    image_ref = final_json["user_inputs"]["image_reference"]
                    content_type = image_ref.get("content_type")
                    # Force JPEG extension
                    extension = ".jpeg"
                    # Sanitize original filename or use a standard name
                    original_filename_stem = pathlib.Path(image_ref.get("filename", "input_image")).stem
                    # Replace spaces or invalid chars in filename stem if needed
                    safe_stem = "".join(c if c.isalnum() or c in ('-', '_') else '_' for c in original_filename_stem)
                    saved_image_filename = f"{safe_stem}_{run_timestamp_str}{extension}"
                    saved_image_path = output_dir / saved_image_filename

                    # Use Pillow to open and save as JPEG, handling transparency
                    img = Image.open(io.BytesIO(image_content_bytes))
                    if img.mode in ('RGBA', 'LA', 'P'):
                        background = Image.new('RGB', img.size, (255, 255, 255))
                        background.paste(img, (0, 0), img.split()[-1] if len(img.split()) == 4 else None)
                        img = background
                    elif img.mode != 'RGB':
                        img = img.convert('RGB')

                    img.save(saved_image_path, format='JPEG', quality=95)
                    print(f"  Input image saved as JPEG to: {saved_image_path}")
                    saved_image_path_str = str(saved_image_path) # Store the full path as string

                except Exception as img_save_error:
                    print(f"  ERROR saving image as JPEG: {img_save_error}")
                    saved_image_filename = None # Reset if saving failed
                    saved_image_path_str = None

            # Prepare JSON for saving (remove base64, add image path)
            json_to_save = json.loads(json.dumps(final_json)) # Deep copy
            if json_to_save.get("user_inputs",{}).get("image_reference"):
                 # Remove base64 regardless of save success
                 if "image_content_base64" in json_to_save["user_inputs"]["image_reference"]:
                      del json_to_save["user_inputs"]["image_reference"]["image_content_base64"]
                 # Add path if image was saved successfully
                 # Store the relative path (just filename) within the output dir
                 json_to_save["user_inputs"]["image_reference"]["saved_image_path"] = saved_image_filename


            # Save JSON
            json_filename = f"{base_filename}.json"
            json_filepath = output_dir / json_filename
            try:
                with open(json_filepath, 'w') as f:
                    json.dump(json_to_save, f, indent=2)
                print(f"  Final JSON saved to: {json_filepath}")
            except Exception as json_save_error:
                 print(f"  ERROR saving JSON: {json_save_error}")


            print("\nProcessing complete.")
            print(f"--- Run FINISHED: {time.strftime('%Y-%m-%d %H:%M:%S')} ---")


        except Exception as e:
            print(f"\n--- UNEXPECTED ERROR during processing ---")
            print(f"Error: {e}")
            print(traceback.format_exc())
            if final_json: # Print the last known state of JSON if an error occurred later
                 print("\n--- Last known JSON state before error ---")
                 # Display without base64
                 final_json_display = json.loads(json.dumps(final_json)) # Deep copy
                 if final_json_display.get("user_inputs",{}).get("image_reference"):
                      if "image_content_base64" in final_json_display["user_inputs"]["image_reference"]:
                           del final_json_display["user_inputs"]["image_reference"]["image_content_base64"]
                 print(json.dumps(final_json_display, indent=2))
            print(f"--- Run ENDED (Error): {time.strftime('%Y-%m-%d %H:%M:%S')} ---")

In [27]:
# @title Step 8: Run Processing Pipeline
# --- Instructions ---
# 1. Ensure you have selected the desired mode and filled in the inputs in Step 2.
# 2. Ensure the LLM Setup (Step 3) has been run and configured correctly if using LLM calls.
# 3. Run THIS cell to execute the pipeline processing steps.

# --- Execution Code ---
run_pipeline_processing()
# Display the output area AFTER the processing function runs
clear_output()
display(output_area)

Output()